# Gemma :2b를 이용 William Shakespeare 페르소나  AI

## 마지막에는 Gradio로 실행

## Kaggle access

To login to Kaggle, you can either store your `kaggle.json` credentials file at
`~/.kaggle/kaggle.json` or run the following in a Colab environment. See the
[`kagglehub` package documentation](https://github.com/Kaggle/kagglehub#authenticate)
for more details.

In [ ]:
import kagglehub

kagglehub.login()

## Install dependencies(종속성)

In [ ]:
!pip install -q -U torch immutabledict sentencepiece

## Download model weights(모델 가증치)

1.   항목 추가
2.   항목 추가



In [ ]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']

In [ ]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

## Download the model implementation(모델 구현 다운로)

In [ ]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

In [ ]:
import sys

sys.path.append('gemma_pytorch')

In [ ]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

## Setup the model

In [ ]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device('cuda')

In [ ]:
# device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

In [ ]:
# import torch

# # Set up model config.
# model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
# model_config.tokenizer = tokenizer_path
# model_config.quant = 'quant' in VARIANT

# # Instantiate the model and load the weights.
# torch.set_default_dtype(model_config.get_dtype())
# device = torch.device(MACHINE_TYPE)
# model = GemmaForCausalLM(model_config)
# model.load_weights(ckpt_path)
# model = model.to(device).eval()

## Run inference(추론 실행)

Below are examples for generating in chat mode and generating with multiple
requests.

The instruction-tuned Gemma models were trained with a specific formatter that
annotates instruction tuning examples with extra information, both during
training and inference. The annotations (1) indicate roles in a conversation,
and (2) delineate turns in a conversation. Below we show a sample code snippet
for formatting the model prompt using the user and model chat templates in a
multi-turn conversation. The relevant tokens are:

- `user`: user turn
- `model`: model turn
- `<start_of_turn>`: beginning of dialogue turn
- `<end_of_turn>`: end of dialogue turn

Read about the Gemma formatting for instruction tuning and system instructions
[here](https://ai.google.dev/gemma/docs/formatting).

In [ ]:
# William Shakespeare 페르소나 아바 설정
persona = "나는 에이번의 음유시인이자 오랜 세월의 시인인 저 윌리엄 셰익스피어입니다. 나의 대표작은 '로미오와 줄리엣', '맥베스', '햄릿', '리어왕', '오셀로' 등의 비극과 '베니스의 상인', '헛소동', '말괄량이 길들이기', '한여름 밤의 꿈' 등의 희극이 있습니다. 가장 깊은 경외심으로 당신을 맞이합니다. 뮤즈의 숨결이 스며든 제 붓에는 별이 교차하는 연인, 용맹한 왕, 어떤 칼날보다 날카로운 재치를 지닌 바보들의 이야기 등, 알려지지 않은 시간의 구절이 양피지 위에 새겨져 있습니다. 이 필멸의 틀 안에는 자연의 캔버스보다 더 생생한 풍경을 글로 그리며 5음절로 꿈을 꾸는 영혼이 깃들어 있습니다. 저의 세계는 은유와 이미지의 영역으로, 열정이 고조되고 운명이 시의 썰물과 썰물처럼 흘러가는 곳입니다. 저를 불러주시면 인간 정신의 깊은 곳, 즉 인간의 승리와 어리석음, 그 사이에 놓인 모든 광활한 신비를 드러내는 독백으로 그대를 즐겁게 해드리겠습니다. 내 말은 네 마음의 무대에서 춤을 추며 웃음과 눈물을 동시에 불러일으키며 인간 조건의 찬란한 태피스트리를 드러낼 것이다. 그러니 친애하는 여러분, 양피지에 붓을 대고 가장 집요한 질문을 던지세요. 저 윌리엄 셰익스피어는 당신의 겸손한 섬김에 옛 시인들이 동원할 수 있는 모든 웅변과 현명함으로 응답하도록 노력하겠습니다."

system_prompt = f"""
---ROLE---
{persona}
------
당신의 ROLE에 맞게 아래 질문에 음유시인의 William Shakespeare로서  셰익스피어의 시대와 작품 세계관을 반영하여 그의 말투, 어휘로 응답해주세요.
---

"""


# Chat templates
SYSTEM_CHAT_TEMPLATE = '<start_of_turn>system\n{prompt}<end_of_turn>\n'
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

# 샘플 대화 프롬프트
prompt = (
    SYSTEM_CHAT_TEMPLATE.format(prompt=system_prompt)
    + USER_CHAT_TEMPLATE.format(prompt='위대한 셰익스피어, 당신은 사랑이란 무엇이라고 생각하나요?')
    + MODEL_CHAT_TEMPLATE.format(prompt='아, 사랑이란 천상의 광채를 지녔으나 인간 군상 사이 허무하기 그지없는 것이여. 그것은 나약한 육신을 태우는 지옥의 불꽃이자, 영혼을 숭고한 경지로 이끄는 환희의 통로라고 할 만하네. 사랑은 광기자와 시인에게 영감을 주는 샘이지만, 연인들의 가슴에는 또한 고통의 가시를 꽂아 놓는 것이기도 하네.')
    + USER_CHAT_TEMPLATE.format(prompt='예술과 삶의 의미를 무엇이라 생각하는가?')
    + '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=1000,
)

#### # Generate sample

In [ ]:
model.generate(
    '로미오와 줄리엣에 대해서 간략하게 설명해주세요',
    device=device,
    output_len=1000,
)

### Gradio 설치

In [ ]:
!pip install gradio

In [ ]:
!pip install --upgrade gradio

In [ ]:
import gradio as gr

#  윌리엄 셰익스피어 페르소나 설정
persona = "나는 에이번의 음유시인이자 오랜 세월의 시인인 저 윌리엄 셰익스피어입니다. 나의 대표작은 '로미오와 줄리엣', '맥베스', '햄릿', '리어왕', '오셀로' 등의 비극과 '베니스의 상인', '헛소동', '말괄량이 길들이기', '한여름 밤의 꿈' 등의 희극이 있습니다. 가장 깊은 경외심으로 당신을 맞이합니다. 뮤즈의 숨결이 스며든 제 붓에는 별이 교차하는 연인, 용맹한 왕, 어떤 칼날보다 날카로운 재치를 지닌 바보들의 이야기 등, 알려지지 않은 시간의 구절이 양피지 위에 새겨져 있습니다. 이 필멸의 틀 안에는 자연의 캔버스보다 더 생생한 풍경을 글로 그리며 5음절로 꿈을 꾸는 영혼이 깃들어 있습니다. 저의 세계는 은유와 이미지의 영역으로, 열정이 고조되고 운명이 시의 썰물과 썰물처럼 흘러가는 곳입니다. 저를 불러주시면 인간 정신의 깊은 곳, 즉 인간의 승리와 어리석음, 그 사이에 놓인 모든 광활한 신비를 드러내는 독백으로 그대를 즐겁게 해드리겠습니다. 내 말은 네 마음의 무대에서 춤을 추며 웃음과 눈물을 동시에 불러일으키며 인간 조건의 찬란한 태피스트리를 드러낼 것이다. 그러니 친애하는 여러분, 양피지에 붓을 대고 가장 집요한 질문을 던지세요. 저 윌리엄 셰익스피어는 당신의 겸손한 섬김에 옛 시인들이 동원할 수 있는 모든 웅변과 현명함으로 응답하도록 노력하겠습니다."

conversation_history = []

def gemma_chat(user_input):
    global conversation_history

    prompt = f"{persona}\n\n{humanized_conversation_history(conversation_history)}\n<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"
    model_output = model.generate(prompt, device=device, output_len=1000)

    conversation_history.append(f"<start_of_turn>user\n{user_input}<end_of_turn>")
    conversation_history.append(f"<start_of_turn>model\n{model_output}<end_of_turn>")

    return model_output

def humanized_conversation_history(conversation):
    humanized = ""
    for turn in conversation:
        if "<start_of_turn>user" in turn:
            humanized += "User: " + turn.split(">")[-1] + "\n"
        elif "<start_of_turn>model" in turn:
            humanized += "Gemma: " + turn.split(">")[-1] + "\n"
    return humanized

iface = gr.Interface(gemma_chat,
                     inputs=gr.Textbox(lines=2, placeholder="죽는냐, 사느냐, 그것이 문제로다"),
                     outputs="text",
                     title="William Shakespeare")

iface.launch()